In [1]:
# | default_exp crawler.article


In [2]:
# | hide
# use with local installs that don't have nbdev

import sys

sys.path.insert(0, "../../")

In [3]:
# | exporti

import datetime as dt
from dataclasses import dataclass, field
import urllib.parse as url_parse

from bs4 import BeautifulSoup
import selenium.webdriver
from selenium.webdriver.common.by import By

# from fastcore.basics import patch_to

import markdownify as md
from dateutil import parser


import datacrew.crawler.crawler as dcc

In [4]:
# | export
@dataclass
class Article:
    url: str
    base_url: str
    entity_base_url: str = None

    url_entity_prefix: str = None
    url_id: str = None

    driver: selenium.webdriver = field(repr=False, default=None)
    soup: BeautifulSoup = field(repr=False, default=None)
    article_soup: BeautifulSoup = field(repr=False, default=None)

    is_success: bool = False

    url_ls: list[str] = field(default_factory=list)
    image_ls: list[str] = field(default_factory=list)

    def __post_init__(self):
        if self.url_entity_prefix[0] != "/":
            self.url_entity_prefix = f"/{self.url_entity_prefix}"

        if self.url_entity_prefix and self.url_entity_prefix in self.base_url:
            self.base_url = self.base_url.replace(self.url_entity_prefix, "")

        self.entity_base_url = url_parse.urljoin(self.base_url, self.url_entity_prefix)

        self.set_id()

        self.get_urls()
    
    def __eq__(self, other):
        return  (self.url_id == other.url_id) or (self.url ==other.url)

    @classmethod
    def get_from_url(
        cls,
        url: str,
        driver: selenium.webdriver,
        base_url: str,
        url_entity_prefix: str = None,
        element_type=By.CLASS_NAME,
        element_id="slds-form-element",
    ):

        soup = dcc.pagesource(
            driver=driver,
            url=url,
            element_type=element_type,
            element_id=element_id,
        )

        return cls(
            url=url,
            url_entity_prefix=url_entity_prefix,
            base_url=base_url,
            soup=soup,
            driver=driver,
        )

    @staticmethod
    def md_soup(soup, **options):
        return md.MarkdownConverter(**options).convert_soup(soup)

    def add_url_to_ls(self, url, is_remove_query_string_parameters: bool = True):
        
        if not self.url_ls:
            self.url_ls = []

        if url.startswith("/") and self.url_entity_prefix in url:
            url = url_parse.urljoin(self.base_url, url)

        if is_remove_query_string_parameters:
            url = url_parse.urljoin(url, url_parse.urlparse(url).path)

        if not url.startswith(self.base_url):
            return

        if url.endswith("/"):
            url = url[:-1]

        if url.split("/")[-4] == "s":
            url = "/".join(url.split("/")[:-1])

        if url not in self.url_ls:
            self.url_ls.append(url)

    def get_urls(self):
        for soup_link in self.soup.find_all("a"):
            url = soup_link.get("href")

            if not url:
                continue

            self.add_url_to_ls(url)
        return self.url_ls

    def set_id(self):
        o = self.url.replace(self.entity_base_url, "")
        self.url_id = o.split("/")[0]
        self.url = url_parse.urljoin(self.entity_base_url, self.url_id)
        return self.url_id

    def get_images(
        self,
        soup=None,  # pass a soup to just exctract images from the selected content.  Default will exctract all images on the page
        test_base_url: str = None,  # pass to limit URLs to a specific base
        debug_prn: bool = False,
    ):
        "extract image urls from soup"

        soup = soup or self.soup

        self.image_ls = [
            {
                "url": f"{self.base_url if item.get('src').startswith('/') else ''}{item.get('src')}",
                "relative_url": item.get("src"),
                "name": item.get("alt"),
            }
            for item in soup.find_all("img")
            if item.get("src", False)
        ]

        if test_base_url:
            self.image_ls = [
                img for img in self.image_ls if img.get("url").startswith(test_base_url)
            ]

        if debug_prn:
            print(self.image_ls)
        return self.image_ls

In [5]:
# | export
class ArticleKB_GetSoupError(Exception):
    def __init__(self, url):
        super().__init__(f"failed to retrieve soup for {url}")


class ArticleKB_ProcessSoupError(Exception):
    def __init__(self, url, search_term):
        super().__init__(f"search term {search_term} does not exist in {url}")


@dataclass(init=False)
class Article_KB(Article):
    title: str = None
    md_str: str = field(default=None, repr=False)
    views: int = None
    created: dt.date = None
    last_updated: dt.date = None
    article_id: str = None

    def __init__(
        self,
        url,
        base_url,
        driver,
        url_entity_prefix="/s/article/",
        debug_prn: bool = False,
    ):
        self.url = url
        self.base_url = base_url

        soup = dcc.pagesource(
            driver=self.driver,
            url=self.url,
            element_type=By.CLASS_NAME,
            element_id="slds-form-element",
            is_return_soup=True,
        )

        super().__init__(
            url=url,
            base_url=base_url,
            soup=soup,
            url_entity_prefix=url_entity_prefix,
            driver=driver,
        )

        if not soup:
            raise ArticleKB_GetSoupError(url=self.url)

        self.article_soup = self.process_soup(soup, debug_prn=debug_prn)
        self.is_success = True

    def process_soup(self, soup: BeautifulSoup, debug_prn: bool = False):
        search_term = "slds-form-element"

        table = soup.find_all(class_=[search_term])

        if not table or table == []:
            raise ArticleKB_ProcessSoupError(url=self.url, search_term=search_term)

        tarticle = []
        for row in table:
            # print("❤️")

            cells = row.find(class_="slds-form-element__label")

            if list(cells.strings):
                content = row.find(class_="slds-form-element__control")
                tarticle.append((list(cells.strings)[0], content))

        kb_soup = dict(tarticle)

        self.title = self.md_soup(kb_soup.get("Title"))
        self.md_str = self.md_soup(kb_soup.get("Article Body"))
        self.article_id = self.md_soup(kb_soup.get("Article Number"))
        self.views = self.md_soup(kb_soup.get("Article Total View Count"))
        self.created = parser.parse(self.md_soup(kb_soup.get("Article Created Date")))

        self.last_updated = parser.parse(
            self.md_soup(kb_soup.get("First Published Date"))
        )

        self.get_images(
            test_base_url="https://domo-support.domo.com/servlet/rtaImage",
            debug_prn=debug_prn,
        )

        return kb_soup

#### sample implementation of article_kb


In [6]:
from selenium.webdriver.common.by import By
import datacrew.crawler.crawler as dcc


TEST_ARTICLE_URL = "https://domo-support.domo.com/s/article/360043429913/"

BASE_URL = "https://domo-support.domo.com"
IMG_BASE_URL = "https://domo-support.domo.com/servlet/rtaImage"

driver = dcc.driversetup(is_headless=False)

test_article = Article_KB(url=TEST_ARTICLE_URL, driver=driver, base_url=BASE_URL)

💤 retrieving https://domo-support.domo.com/s/article/360043429913/ 💤


In [7]:
for key in test_article.__dict__.keys():
    if key in ["soup", "article_soup", "driver", "md_str"]:
        continue
    print({key: getattr(test_article, key)})

{'url': 'https://domo-support.domo.com/s/article/360043429913'}
{'base_url': 'https://domo-support.domo.com'}
{'entity_base_url': 'https://domo-support.domo.com/s/article/'}
{'url_entity_prefix': '/s/article/'}
{'url_id': '360043429913'}
{'is_success': True}
{'url_ls': ['https://domo-support.domo.com/s/knowledge-base', 'https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC', 'https://domo-support.domo.com/s/article/360042924034', 'https://domo-support.domo.com/s/article/360043429933', 'https://domo-support.domo.com/s/article/360043429953', 'https://domo-support.domo.com/s/article/360042924094', 'https://domo-support.domo.com/s/article/360043429993', 'https://domo-support.domo.com/s/article/360043428753', 'https://domo-support.domo.com/s/article/360043429913', 'https://domo-support.domo.com/s/article/360042925494', 'https://domo-support.domo.com/s/article/360043429693']}
{'image_ls': [{'url': 'https://domo-support.domo.com/servlet/rtaImage?eid=ka05w000001248e&feoid=00N5w00000Ri7BU&re

In [8]:
# | export


@dataclass(init=False)
class Article_Category(Article):

    category: str = None
    category_description: str = None

    is_child_recursive: bool = True
    crawl_category_id_ls: [str] = field(default=None)

    child_category_ls: list[dict] = field(default_factory=[])
    child_article_ls: list[dict] = field(default_factory=[])

    def __init__(
        self,
        url,
        base_url,
        url_entity_prefix="s/topic/",
        crawl_category_id_ls=None,
        debug_prn: bool = False,
        driver=None,
        is_child_recursive: bool = False,
    ):

        self.child_article_ls = []
        self.child_category_ls = []
        self.crawl_category_id_ls = crawl_category_id_ls or []
        self.is_child_recursive = is_child_recursive

        if not driver:
            driver = dcc.driversetup(is_headless=False)

        soup = dcc.pagesource(
            driver=driver,
            url=url,
            element_type=By.CLASS_NAME,
            element_ls=["section-list-item", "article-list-item"],
            is_return_soup=True,
        )

        if not soup:
            raise ArticleKB_GetSoupError(url=url)

        super().__init__(
            url=url,
            base_url=base_url,
            soup=soup,
            url_entity_prefix=url_entity_prefix,
            driver=driver,
        )

        self.article_soup = self.process_soup(soup, debug_prn=debug_prn)
        self.is_success = True

        
    def process_soup(self, soup: BeautifulSoup, debug_prn: bool = False):
        # process parent attributes

        article_soup = soup.find(class_=["cDomoKBCategoryNav"])

        category_soup = article_soup.find("h1")
        self.category = category_soup.get_text()

        category_description_soup = article_soup.find("p")
        self.category_description = (
            category_description_soup and category_description_soup.get_text()
        )

        table_item_term = ["section-list-item", "article-list-item"]
        table_soup = article_soup.find_all(class_=[table_item_term])

        if not table_soup or table_soup == []:
            raise ArticleKB_ProcessSoupError(url=self.url, search_term=table_item_term)

        # process children
        for row in table_soup:
            url = row.find("a").get("href")
            if url[0] == "/":
                url = url_parse.urljoin(self.base_url, url)

            child_id = url.split("/")[-1]

            print(f"❤️ child url - {url}, {child_id}")

            self.add_url_to_ls(url)

            # if article
            if "/s/article/" in url:
                self.child_article_ls.append(url)
                continue

            # if anything else
            if self.url_entity_prefix not in url:
                continue

            # if category
            if (
                self.url_entity_prefix in url
                and child_id not in self.crawl_category_id_ls
            ):
                self.crawl_category_id_ls.append(child_id)

                child_obj = {"category": row.get_text(), "id": child_id, "url": url}

                if self.is_child_recursive and self.url_entity_prefix in url:

                    child_obj.update(
                        {
                            "child_article": Article_Category(
                                url=url,
                                base_url=self.base_url,
                                crawl_category_id_ls=self.crawl_category_id_ls,
                                url_entity_prefix=self.url_entity_prefix,
                                driver=self.driver,
                            )
                        }
                    )

                self.child_category_ls.append(child_obj)

        return {
            "category": category_soup,
            "description": category_description_soup,
            "children": table_soup,
        }

In [9]:
TEST_TOPIC_URL = 'https://domo-support.domo.com/s/topic/0TO5w000000ZamoGAC'
# TEST_TOPIC_URL = "https://domo-support.domo.com/s/topic/0TO5w000000ZanAGAS/beast-mode"

BASE_URL = "https://domo-support.domo.com"

IMG_BASE_URL = "https://domo-support.domo.com/servlet/rtaImage"

OUTPUT_FOLDER = "../../raw_kb"

driver = dcc.driversetup(is_headless=False)

test_article = Article_Category(url=TEST_TOPIC_URL, driver=driver, base_url=BASE_URL)

💤 retrieving https://domo-support.domo.com/s/topic/0TO5w000000ZamoGAC 💤
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZanDGAS, 0TO5w000000ZanDGAS
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZaoEGAS, 0TO5w000000ZaoEGAS
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZaoNGAS, 0TO5w000000ZaoNGAS
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZanwGAC, 0TO5w000000ZanwGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZanZGAS, 0TO5w000000ZanZGAS
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000Zao4GAC, 0TO5w000000Zao4GAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZaoHGAS, 0TO5w000000ZaoHGAS
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZanAGAS, 0TO5w000000ZanAGAS
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZdTCGA0, 0TO5w000000ZdTCGA0


In [10]:
test_article.url_ls

['https://domo-support.domo.com/s/knowledge-base',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZanDGAS',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZaoEGAS',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZaoNGAS',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZanwGAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZanZGAS',
 'https://domo-support.domo.com/s/topic/0TO5w000000Zao4GAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZaoHGAS',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZanAGAS',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZdTCGA0']

In [11]:
# | export

@dataclass(init=False)
class Article_KB_Home(Article):

    category: str = None
    category_description: str = None

    is_child_recursive: bool = True
    crawl_category_id_ls: [str] = field(default=None)

    child_category_ls: list[dict] = field(default_factory=[])
    child_article_ls: list[dict] = field(default_factory=[])

    def __init__(
        self,
        url,
        base_url,
        url_entity_prefix="s/knowledge-base",
        crawl_category_id_ls=None,
        debug_prn: bool = False,
        driver=None,
        is_child_recursive: bool = False,
    ):

        self.child_article_ls = []
        self.child_category_ls = []
        self.crawl_category_id_ls = crawl_category_id_ls or []
        self.is_child_recursive = is_child_recursive

        if not driver:
            driver = dcc.driversetup(is_headless=False)

        soup = dcc.pagesource(
            driver=driver,
            url=url,
            element_type=By.CLASS_NAME,
            element_ls=["blocks-list"],
            is_return_soup=True,
        )

        if not soup:
            raise ArticleKB_GetSoupError(url=url)

        super().__init__(
            url=url,
            base_url=base_url,
            soup=soup,
            url_entity_prefix=url_entity_prefix,
            driver=driver,
        )

        self.article_soup = self.process_soup(soup, debug_prn=debug_prn)
        self.is_success = True

        
    def process_soup(self, soup: BeautifulSoup, debug_prn: bool = False):
        # process parent attributes

        article_soup = soup.find(class_=["cDomoKBCategoryNav"])

        category_soup = article_soup.find("h1")
        self.category = category_soup.get_text()

        category_description_soup = article_soup.find("p")
        self.category_description = (
            category_description_soup and category_description_soup.get_text()
        )

        table_item_term = ["blocks-item"]
        table_soup = article_soup.find_all(class_=[table_item_term])

        if not table_soup or table_soup == []:
            raise ArticleKB_ProcessSoupError(url=self.url, search_term=table_item_term)

        # process children
        for row in table_soup:
            url = row.find("a").get("href")
            if url[0] == "/":
                url = url_parse.urljoin(self.base_url, url)

            child_id = url.split("/")[-1]

            print(f"❤️ child url - {url}, {child_id}")

            self.add_url_to_ls(url)

            # if article
            if "/s/article/" in url:
                self.child_article_ls.append(url)
                continue

            # if anything else
            if '/s/topic/' in url and child_id not in self.crawl_category_id_ls : 
            
                self.crawl_category_id_ls.append(child_id)

                child_obj = {"category": row.get_text(), "id": child_id, "url": url}

                if self.is_child_recursive and self.url_entity_prefix in url:

                    child_obj.update(
                        {
                            "child_article": Article_Category(
                                url=url,
                                base_url=self.base_url,
                                crawl_category_id_ls=self.crawl_category_id_ls,
                                url_entity_prefix=self.url_entity_prefix,
                                driver=self.driver,
                            )
                        }
                    )

                self.child_category_ls.append(child_obj)

        return {
            "category": category_soup,
            "description": category_description_soup,
            "children": table_soup,
        }

In [12]:
TEST_HOME_URL = "https://domo-support.domo.com/s/knowledge-base"

BASE_URL = "https://domo-support.domo.com"

IMG_BASE_URL = "https://domo-support.domo.com/servlet/rtaImage"

OUTPUT_FOLDER = "../../raw_kb"

driver = dcc.driversetup(is_headless=False)

test_home = Article_KB_Home(url=TEST_HOME_URL, driver=driver, base_url=BASE_URL)

💤 retrieving https://domo-support.domo.com/s/knowledge-base 💤
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamsGAC, 0TO5w000000ZamsGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC, 0TO5w000000ZammGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamzGAC, 0TO5w000000ZamzGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamoGAC, 0TO5w000000ZamoGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamnGAC, 0TO5w000000ZamnGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamlGAC, 0TO5w000000ZamlGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZamqGAC, 0TO5w000000ZamqGAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000Zan0GAC, 0TO5w000000Zan0GAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000Zan2GAC, 0TO5w000000Zan2GAC
❤️ child url - https://domo-support.domo.com/s/topic/0TO5w000000ZampGAC, 0TO5w000000ZampGAC
❤️ child url - htt

In [13]:
test_home.child_category_ls
test_home.url_ls

['https://domo-support.domo.com/s/knowledge-base',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamsGAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZammGAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamzGAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamoGAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamnGAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamlGAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamqGAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000Zan0GAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000Zan2GAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZampGAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000Zan1GAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamyGAC']

In [14]:
# | hide
import nbdev

nbdev.nbdev_export()

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 6384: character maps to <undefined>